In [ ]:
pip install -r requirements.txt

In [1]:
import boto3
import json
from datetime import datetime
from dynaconf import Dynaconf
from evaluate import load
from tqdm import tqdm
import pandas as pd

from src.completion import get_completion

In [ ]:
settings = Dynaconf(settings_files=["settings.toml"], env="haiku-3", environments=True)
bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = settings.region)

## Translation – spa-eng

In [13]:
eng = []
with open('data/spa-eng/eng-10.txt', 'r') as f:
    for l in f:
        eng.append(l)

spa = []
with open('data/spa-eng/spa-10.txt', 'r') as f:
    for l in f:
        spa.append(l)

In [19]:
sys_prompt = "You are a translator from Spanish to English. Only provide the translation."

In [20]:
# Load BLEU metric
bleu = load("bleu")

# Generate predictions
predictions = [get_completion(src, sys_prompt) for src in tqdm(spa)]

# Evaluate with BLEU
results = bleu.compute(predictions=predictions, references=eng)
print("BLEU Score:", results["bleu"])


100%|██████████| 10/10 [00:07<00:00,  1.31it/s]

BLEU Score: 0.5421764828099125


In [22]:
d = pd.DataFrame({
    "source": spa, 
    "predictions": predictions, 
    "references": eng
})
print(d.to_latex(index=False))

/tmp/ipykernel_27346/4180831044.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d.to_latex(index=False))


\begin{tabular}{lll}
\toprule
                                            source &                                        predictions &                                         references \\
\midrule
Se divulga un compuesto en donde, A o A represe... & The compound is disclosed where A or A represen... & Disclosed is a compound in which, Aa or Ab repr... \\
[0036] En los dibujos, los elementos iguales o ... & [0036] In the drawings, equal or equivalent ele... & In the drawings, the same or equivalent element... \\
La Figura 8 muestra un diagrama de flujo que il... & The Figure 8 shows a flow diagram that illustra... & Fig. 8 shows a flowchart illustrating a method ... \\
Las bombasde ondas lambda (LW1, LW2, LW3) y el ... & The lambda wave pumps (LW1, LW2, LW3) and the l... & The Lamb-wave pumps (LW1, LW2, LW3) and Lamb-wa... \\
Se proporcionan mas detalles, por ejemplo, en H... & More details are provided, for example, in Held... & Further details are provided, e.g., by Held et ... \\
N

In [23]:
eng = []
with open('data/spa-eng/eng-3000.txt', 'r') as f:
    for l in f:
        eng.append(l)

spa = []
with open('data/spa-eng/spa-3000.txt', 'r') as f:
    for l in f:
        spa.append(l)

In [ ]:
# Load BLEU metric
bleu = load("bleu")

# Generate predictions
predictions = [get_completion(src, sys_prompt) for src in tqdm(spa)]

# Evaluate with BLEU
results = bleu.compute(predictions=predictions, references=eng)
print("BLEU Score:", results["bleu"])


 99%|█████████▊| 2959/3000 [32:47<00:19,  2.08it/s]

In [25]:
results

{'bleu': 0.5513007663279937,
 'precisions': [0.7847464303298868,
  0.6047590055809234,
  0.4877290661531538,
  0.3990837583196473],
 'brevity_penalty': 1.0,
 'length_ratio': 1.031519497800847,
 'translation_length': 101550,
 'reference_length': 98447}